In [1]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.0.1


In [2]:
sess_config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth=True))

In [4]:
sess = tf.Session(config = sess_config)

a = tf.ones((2,2))
b = tf.reduce_sum(a, axis=1)
c = tf.reshape(a, (1,4))

print(a)
print(sess.run(a))
print(sess.run(b))
print(sess.run(c))

sess.close()

Tensor("ones_1:0", shape=(2, 2), dtype=float32)
[[ 1.  1.]
 [ 1.  1.]]
[ 2.  2.]
[[ 1.  1.  1.  1.]]


In [13]:
bb = tf.constant([[2,2]], dtype=tf.float32)
with tf.Session(config = sess_config) as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(b*a))
    print(bb.shape, " = " ,  a.shape)
    print(sess.run(bb*a))
    print(sess.run(tf.matmul(bb,a)))

[[ 1.  1.]
 [ 1.  1.]]
[ 2.  2.]
[[ 2.  2.]
 [ 2.  2.]]
(1, 2)  =  (2, 2)
[[ 2.  2.]
 [ 2.  2.]]
[[ 4.  4.]]


In [16]:
w1 = tf.Variable(tf.random_normal([5,2], stddev=0.1))
w2 = tf.Variable([[3,2,3],[1,2,3]])

with tf.Session(config = sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    w1_fetch, w2_fetch = sess.run([w1,w2])
    print(w1_fetch)
    print(w2_fetch)
    print(w1)
    print(w2)
    

[[ 0.01221879 -0.04479288]
 [-0.09659357 -0.02552533]
 [ 0.06752612 -0.07375654]
 [ 0.06802593 -0.03256749]
 [-0.05811738 -0.1637394 ]]
[[3 2 3]
 [1 2 3]]
Tensor("Variable/read:0", shape=(5, 2), dtype=float32)
Tensor("Variable_1/read:0", shape=(2, 3), dtype=int32)


In [18]:
state = tf.Variable(0, name = "counter")
update = tf.assign(state, tf.add(state, tf.constant(1)))

with tf.Session(config = sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    print("init : ", sess.run(state))
    for i in range(3):
        sess.run(update)
        print("update {} : ".format(i), sess.run(state))
        

init :  0
update 0 :  1
update 1 :  2
update 2 :  3


In [20]:
matrix1 = tf.placeholder(tf.float32, [2,2])
matrix2 = tf.placeholder(shape=[2,2], dtype=tf.float32)
product = tf.matmul(matrix1, matrix2)

with tf.Session(config=sess_config) as sess:
    mv1 = sess.run(tf.random_uniform(shape=[2,2], dtype=tf.float32))
    mv2 = sess.run(tf.random_uniform(shape=[2,2], dtype=tf.float32))
    result = sess.run(product, feed_dict={matrix1:mv1, matrix2:mv2})
    print (mv1)
    print (mv2)
    print(result)
    

[[ 0.19563007  0.42904913]
 [ 0.76093233  0.82296515]]
[[ 0.25382924  0.18190956]
 [ 0.80656767  0.97068322]]
[[ 0.39571378  0.45205778]
 [ 0.85692394  0.93725932]]


In [26]:
and_x = np.array([[0,0], [0,1], [1,0],[1,1]])
and_y = np.array([0,0,0,1])
or_y = np.array([0,1,1,1])

learning_rate = 0.1

inputs = tf.placeholder(tf.float32, [None, 2])
label = tf.placeholder(tf.int64, [None])
w  = tf.Variable(tf.random_uniform([2,2]))
b = tf.Variable(tf.random_uniform([2]))

logit = tf.matmul(inputs, w)+b
pred_op = tf.nn.softmax(logit)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit))
opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
#opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1000):
        _, loss = sess.run([opt, loss_op], feed_dict={inputs: and_x, label:and_y})
        if (step +1) % 100 == 0:
            pred = sess.run(pred_op,  feed_dict={inputs: and_x, label:and_y})
            print ("step", step+1, " loss : ", loss, " gt : ", and_y, " predict : ", np.argmax(pred, axis=1))

step 100  loss :  0.0257014  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 200  loss :  0.0103153  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 300  loss :  0.00582025  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 400  loss :  0.00382112  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 500  loss :  0.00273462  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 600  loss :  0.00206929  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 700  loss :  0.00162808  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 800  loss :  0.00131831  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 900  loss :  0.00109132  gt :  [0 0 0 1]  predict :  [0 0 0 1]
step 1000  loss :  0.000919332  gt :  [0 0 0 1]  predict :  [0 0 0 1]


In [29]:
xor_y = np.array([1,0,0,1])

W1 = tf.Variable(tf.random_uniform([2,2]))
W2 = tf.Variable(tf.random_uniform([2,2]))
b1 = tf.Variable(tf.zeros([2]))
b2 = tf.Variable(tf.zeros([2]))

layer1 = tf.matmul(inputs, W1)+b1
layer1 = tf.sigmoid(layer1)
logit = tf.matmul(layer1, W2)+b2
pred_op = tf.nn.softmax(logit)

loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = label, logits = logit))
#opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)
opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_op)

with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10000):
        _, loss = sess.run([opt, loss_op], feed_dict={inputs: and_x, label:xor_y})
        if (step +1) % 1000 == 0:
            pred = sess.run(pred_op,  feed_dict={inputs: and_x, label:xor_y})
            print ("step", step+1, " loss : ", loss, " gt : ", xor_y, " predict : ", np.argmax(pred, axis=1))
            

step 1000  loss :  0.686096  gt :  [1 0 0 1]  predict :  [1 1 0 0]
step 2000  loss :  0.581726  gt :  [1 0 0 1]  predict :  [1 0 0 0]
step 3000  loss :  0.233252  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 4000  loss :  0.0605868  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 5000  loss :  0.0304698  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 6000  loss :  0.0198136  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 7000  loss :  0.0145339  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 8000  loss :  0.0114199  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 9000  loss :  0.00937839  gt :  [1 0 0 1]  predict :  [1 0 0 1]
step 10000  loss :  0.00794195  gt :  [1 0 0 1]  predict :  [1 0 0 1]


In [30]:
print("bye bye")

bye bye
